# Sphinx API documentation

[Sphinx](https://www.sphinx-doc.org) is a powerful documentation tool, based on [reStructuredText](https://www.sphinx-doc.org/en/3.x/usage/restructuredtext/basics.html).

If you are more comfortable with markdown, you can use [MyST](https://myst-parser.readthedocs.io)


In [1]:
ls

env-1                   pipfile       pip-tools         sphinx-docs.ipynb
environments.ipynb      Pipfile       requirements.txt
from-script-to-project  Pipfile.lock  sphinx-docs


In [2]:
cd sphinx-docs

In [3]:
ls

docs  docs-2023  docs-done  package_to_document  __pycache__


We can initialize our docs with the command:

    sphinx-quickstart

which will prompt with a few questions to set up the docs directory.

There are two key files created (and several more):

1. `conf.py` - the sphinx configuration file
2. `index.rst` the first page of our documentation. This is the landing page for our docs

Let's start by checking out the docs:

    make html

in the docs directory, then open `_build/index.html`


The first thing we are going to do is add the [autodoc](https://www.sphinx-doc.org/en/master/usage/extensions/autodoc.html) extension.
`autodoc` is an extension that defines **directives** (commands) for automatically documenting your code.

We add this extension to the `conf.py` file

```python
extensions = [
    "sphinx.ext.autodoc",
]
```

Our module `myarray` defines an `Array` class and an `asarray` function. We want to docment these.
Since we went through the trouble of adding docstrings, it would be nice to import those into our docs! That's what autodoc is for.

Inspecting our module and class, we can see their docstrings:


In [4]:
conda activate UIO-IN3110
python3 -c "import myarray; help(myarray)"

(UIO-IN3110) 
Help on package myarray:

NAME
    myarray

PACKAGE CONTENTS
    myarray

CLASSES
    builtins.object
        myarray.myarray.Array
    
    class Array(builtins.object)
     |  Array(shape, *values)
     |  
     |  A class defining an API for arrays
     |  
     |  Methods defined here:
     |  
     |  __add__(self, other)
     |      Element-wise adds Array with another Array or number.
     |      
     |      If the method does not support the operation with the supplied arguments
     |      (specific data type or shape), it should return NotImplemented.
     |      
     |      Args:
     |          other (Array, float, int): The array or number to add element-wise to this array.
     |      
     |      Returns:
     |          Array: the sum as a new array.
     |  
     |  __eq__(self, other)
     |      Compares an Array with another Array.
     |      
     |      If the two array shapes do not match, it should return False.
     |      If `other` is an unex

In [5]:
python3 -c "import myarray; help(myarray.Array.mean)"

Help on function mean in module myarray.myarray:

mean(self)
    Computes the mean of the array
    
    Only needs to work for numeric data types.
    
    Returns:
        float: The mean of the array values.

(UIO-IN3110) 


Let's get started by creating a new file int he `source` directory, with `source/api.rst`:

```rst
My Array API
============

.. automodule:: myarray

```

and we can try to build the documentation again with `make html`.
We get the following warning

> ```
> docs/api.rst: WARNING: document isn't included in any toctree
> ```


```{note}
Inspect the package `myarray`, both `__init__.py` and `myarray.py` contains `__all__ = [...]` statements.
This is to instruct Python (and autodoc) what members should be included in an import and in documentation.
As we have created the `myarray`-package, the module is described by what is in the `__init__.py` file. We could also document the module `myarray.myarray` explicitly.

```


In [6]:
head -n5 package_to_document/src/myarray/__init__.py

from myarray.myarray import Array, asarray

__all__ = ["Array", "asarray"](UIO-IN3110) 


In [7]:
head -n8 package_to_document/src/myarray/myarray.py

"""
Defines an Array class.

Class template for Arrays, IN 3110 assignment 3
"""

__all__ = ["Array", "asarray"]

(UIO-IN3110) 


The toctree (**t**able **o**f **c**ontents tree) warning is because our new `api.rst` document is not included in any `.. toc::` directives.
Sphinx wants to make sure that you can navigate through all the documentation
via table of contents, so it encourages you to include all your documents in

We can add our new document to the top-level table of contents in `index.rst`:

```rst
.. toctree::
   :maxdepth: 2
   :caption: Contents:

   api.rst
```


No we can build again with `make html` and open `_build/html/api.html`.

```{note}
To build the documentation for a module, it has to be installed on your system.
As `myarray` is a proper package, we use `python3 -m pip install /path/to/myarray` to install it prior to building with sphinx.

If you really want to avoid making a package, one can modify the `sys.path` inside the `conf.py` file.
However, this is not recommended!

```

Next, we can add our function to the documentation:


```rst
My Array API
============

.. automodule:: myarray

.. autofunction:: asarray
```

and build again with `make html`, reloading [\_build/html/api.html](sphinx-docs/docs/_build/html/api.html)


Similarly, we can document our `Array` class:

```rst
My Array API
============

.. automodule:: myarray

.. autofunction:: asarray

.. autoclass:: Array
```

now all our top-level items are documented, but we probably want to see our member methods, not just the constructor.


For that, we add the `:members:` option to autoclass:

```rst
My Array API
============

.. automodule:: myarray

.. autofunction:: asarray

.. autoclass:: Array
    :members:
```


And finally, since we are most interested in "special" methods (`__mul__` and friends), we may want to document the special methods as well, which are not included by default in `:members:`:

```rst
My Array API
============

.. automodule:: myarray

.. autofunction:: asarray

.. autoclass:: Array
    :members:
    :special-members:
```


Now we are done setting up our API docs.
[autodoc](https://www.sphinx-doc.org/en/3.x/usage/extensions/autodoc.html) has lots of options for controlling which members are

The last step we are going to do is enable another extension called [napoleon](https://www.sphinx-doc.org/en/3.x/usage/extensions/napoleon.html),
which understands special docstring formats, which are widely used.
These give nice highlighting and linking for parameters, return types, etc.

```python
extensions = [
    "sphinx.ext.autodoc",
    "sphinx.ext.napoleon",
]
```

Now rebuild, and see how much nicer the output is!
